In [1]:
# Hi, this is an AI agent that generate radiology reports from adult glioma patients MRI and tabular data .

# At frist step importing essentail libraries 
!pip install SimpleITK
!pip install nibabel --quiet





import SimpleITK as sitk
import nibabel as nib
import numpy as np
import pandas as pd 
import tensorflow as tf 

import keras
import os 
import cv2
import glob 
import pathlib as Path
import matplotlib.pyplot as plt
import re 
import tensorflow_datasets as tfds
import random
from keras import regularizers
from keras.models import Model

from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization , Input , Reshape 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import plot_model
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from keras import layers
from collections import defaultdict
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision
import tqdm as tqdm

# By seting on mixed_float16 , reduce the computational cost 
mixed_precision.set_global_policy("mixed_float16")



2025-09-03 14:17:55.276607: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756909075.495443      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756909075.566937      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# At second step we design a dataset preprocessing pipline for 3D segmentation model
image_data_folders = ["/kaggle/input/glioma-train-test/train data"]
patient_files = defaultdict(list)
for folder in image_data_folders :
    for root,dirs,files in os.walk(folder):
        for file in files :
            if file.endswith('.nii') or file.endswith('.nii.gz'):
                patient_id = '_'.join(file.split('_')[:1])
                patient_path = os.path.join(root,file)
                patient_files[patient_id].append(patient_path)




def nifti_loader_3D (path, expected_dim = 3 , min_size = (112,112,80)):
    try:
        img = sitk.ReadImage(path)

    except Exception as e :
        raise ValueError (f'cannot read {path}:{e}')

    array = array = sitk.GetArrayFromImage(img)

    if array.ndim != expected_dim :
        raise ValueError (f'unexpected dimention  {array.ndim} in {path}')

    if any(s<m for s , m in zip(array.shape,min_size)):
        raise ValueError (f'unexpected size  {array.shape} in {path}')

    if not np.isfinite(array).all():
        raise ValueError (f'unexpected value  in {path}')
    return img  





def resampler (image , spacing = (2,2,2), origin = None , direction = None ,is_label = False ):
    image_size = image.GetSize()
    image_spacing = image.GetSpacing()
    final_size = [int(np.round(image_size[i]*(image_spacing[i]/spacing[i]))) for i in range(3) ]
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(spacing)
    resampler.SetSize(final_size)
    if direction is None : 
        resampler.SetOutputDirection(image.GetDirection())
    else :
        resampler.SetOutputDirection(direction)
    if origin is None :
        resampler.SetOutputOrigin(image.GetOrigin())
    else :
        resampler.SetOutputOrigin(origin)

    
    resampler.SetTransform(sitk.Transform())

    if is_label : 
        resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)

    return resampler.Execute(image)







def crop_pad(image, target_size=(224, 224, 160 ), pad_value=0):

    image_size = list(image.GetSize())  
    target_size = list(target_size)
    
    start_index = []
    size_after_crop = []
    pad_lower = []
    pad_upper = []
    
    for dim in range(3):
        if image_size[dim] >= target_size[dim]:
            # crop center
            start = (image_size[dim] - target_size[dim]) // 2
            start_index.append(start)
            size_after_crop.append(target_size[dim])
            pad_lower.append(0)
            pad_upper.append(0)
        else:
           
            start_index.append(0)
            size_after_crop.append(image_size[dim])
            total_pad = target_size[dim] - image_size[dim]
            lower = total_pad // 2
            upper = total_pad - lower
            pad_lower.append(lower)
            pad_upper.append(upper)

    
    cropped = sitk.RegionOfInterest(image, size_after_crop, start_index)
    
    padded = sitk.ConstantPad(cropped, pad_lower, pad_upper, pad_value)

    return padded


def rescaling(image):
    p1 = np.percentile(image,1)
    p99 = np.percentile(image,99)
    image = np.clip(image,p1,p99)
    img = (image - np.mean(image)) / (np.std(image) + 1e-8)
    return img



def sitk_to_numpy(image):
    array = sitk.GetArrayFromImage(image)
    array = np.expand_dims(array,axis = 0)
    return array.astype(np.float32)

def load_and_modify_patients(patient_id,patient_files):
    paths =  patient_files.get (patient_id,[])
    image_modalities = {}
    for path in paths :
        if not os.path.exists(path) or os.path.getsize(path) < 1000:
            continue
        file_name = path.lower()
        if 't1c' in file_name :
            image_modalities['t1c'] = nifti_loader_3D(path)
        elif 'flair' in file_name :
            image_modalities['flair'] = nifti_loader_3D(path)
        elif 'segmentation' in file_name :
            image_modalities['segmentation'] = nifti_loader_3D(path)
    required_modalities = ['t1c','flair','segmentation']
    if not all( k in image_modalities for k in required_modalities ):
        raise ValueError (f"Incomplete data for patient {patient_id}")

    t1c_image = resampler(crop_pad (image_modalities['t1c']), is_label = False)
    t2f_image = resampler(crop_pad (image_modalities['flair']), is_label = False)
    mask = resampler(crop_pad (image_modalities['segmentation']), is_label = True)

    t1c_image = sitk_to_numpy(t1c_image).squeeze(0)
    t2f_image = sitk_to_numpy(t2f_image).squeeze(0)
    mask = sitk_to_numpy(mask).squeeze(0)
    if not (t1c_image.shape == t2f_image.shape == mask.shape) :
        raise ValueError (f"shape mismatch for patient {patient_id}:" 
        f"T1c {img_t1c.shape}, T2f {img_t2f.shape}, Mask {mask.shape}")
    t1c_image = rescaling(t1c_image)
    t2f_image = rescaling(t2f_image)    
    image = np.stack([t1c_image, t2f_image], axis=0)  # [C, D, H, W]
    image = np.nan_to_num(image)

    # Intensity checks
    if np.max(image) == 0:
        raise ValueError(f"All-zero image for patient {patient_id}")
    if np.std(image) < 1e-6:
        raise ValueError(f"Flat image (no variation) for patient {patient_id}")

    
    
    image = np.transpose(image, (3, 2, 1, 0))  
    mask = np.transpose(mask, (2,1,0)) 
    # --- Process mask ---
    unique_labels = np.unique(mask)
    if not np.all(np.isin(unique_labels, [0, 1, 2, 4])):
        raise ValueError(f"Unexpected labels {unique_labels} in mask for patient {patient_id} ")
        
    mask[mask == 4] = 3
    
    mask = np.clip(mask, 0, 3)
    mask = np.nan_to_num(mask)

    if np.sum(mask) == 0:
        raise ValueError(f"Empty mask for patient {patient_id}")

    mask_resized = tf.one_hot(mask.astype(np.int32),
                              depth=4,
                              on_value=1.0,
                              off_value=0.0)

    return image.astype(np.float32), mask_resized

In [3]:

all_ids = list(patient_files.keys())
split_idx = int(0.8 * len(all_ids))   
train_ids = all_ids[:split_idx]
val_ids   = all_ids[split_idx:]





def augment_3d_mri(img, mask, flip_prob=0.4,  noise_prob=0.4, noise_std=0.01):

   
    if tf.random.uniform(()) < flip_prob:
        axis = tf.random.uniform([], minval=0, maxval=3, dtype=tf.int32)  # only spatial axes
        img  = tf.reverse(img, axis=[axis])
        mask = tf.reverse(mask, axis=[axis])


   
    if tf.random.uniform(()) < noise_prob:
        noise = tf.random.normal(shape=tf.shape(img), mean=0.0, stddev=noise_std, dtype=img.dtype)
        img = img + noise

    return img, mask












def generator(patient_ids):
    for pid in patient_ids:
        try:
            img, mask = load_and_modify_patients(pid, patient_files)
            yield img, mask
        except ValueError as e:
            print(f"Skipping {pid}: {e}")
            continue

train_dataset = (
    tf.data.Dataset.from_generator(
        lambda: generator(train_ids),
        output_signature=(
            tf.TensorSpec(shape=(112,112,80,2), dtype=tf.float32),
            tf.TensorSpec(shape=(112,112,80,4), dtype=tf.float32),
        )
    )
    .map(lambda x, y: augment_3d_mri(x, y, flip_prob=0.3, noise_prob=0.3, noise_std=0.01),
         num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(buffer_size=300)
    .repeat()
    .batch(2)
    .prefetch(8)
)



val_dataset = (
    tf.data.Dataset.from_generator(
        lambda: generator(val_ids),
        output_signature=(
            tf.TensorSpec(shape=(112,112,80, 2), dtype=tf.float32),
            tf.TensorSpec(shape=(112,112,80, 4), dtype=tf.float32),
        )
    )
    .repeat()
    .batch(2)
    .prefetch(8)
)

I0000 00:00:1756909099.827400      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [4]:
def dice_loss(y_true, y_pred):
    smooth = 1e-6
    
    
    intersection = tf.reduce_sum(y_true * y_pred, axis=[0,1,2,3])
    union = tf.reduce_sum(y_true, axis=[0,1,2,3]) + tf.reduce_sum(y_pred, axis=[0,1,2,3])
    
  
    dice_per_class = (2.0 * intersection + smooth) / (union + smooth)
    
   
    class_weights = tf.constant([0.000742, 0.6551, 0.0873, 0.2568], dtype=tf.float32)
    
  
    weighted_dice = tf.reduce_sum(class_weights * dice_per_class) / tf.reduce_sum(class_weights)
    
    return 1.0 - weighted_dice

def combined_loss(y_true,y_pred):
    categorical_focal_crossentropy =tf.keras.losses.CategoricalFocalCrossentropy( alpha=[0.000742, 0.6551, 0.0873, 0.2568]
, gamma=2.0, from_logits=False, label_smoothing=0.0,axis=-1
                                                                                 ,reduction='sum_over_batch_size',name='categorical_focal_crossentropy')(y_true, y_pred)
    Dice_loss = dice_loss(y_true, y_pred)
  
    return (categorical_focal_crossentropy* 1)+ (Dice_loss*2)

iou = tf.keras.metrics.IoU(
    4,
    [1,2,3],
    name='iou',
    dtype=None,
    ignore_class=None,
    sparse_y_true=False,
    sparse_y_pred=False,
    axis=-1 )

In [5]:

def sensory_neuron (primary_filters, secondry_filters ,kernel_size , dilation , reciver ) :
    a = reciver
    a = layers.Conv3D(primary_filters, kernel_size , padding='same',dilation_rate = dilation, kernel_initializer='he_normal', activation='relu')(a)
    a_2 = layers.Conv3D(secondry_filters , kernel_size, padding='same',dilation_rate = dilation, kernel_initializer='he_normal', activation='relu')(a)
    a_3 = BatchNormalization()(a_2)
    a_4 = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(a_3)
    return a_4



def laterel_geniculate_body (filters ,kernel_size ,dilation, reciver) : 
    f = reciver 
    f_1a = layers.Conv3D(filters , kernel_size , padding='same', dilation_rate = dilation , kernel_initializer='he_normal', activation='relu')(f)
    f_1b = BatchNormalization()(f_1a)
    f_1c = layers.MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding="same")(f_1b)
    return f_1c



    




def motor_neuron (primary_filters, secondry_filters ,primary_kernel_size,secondry_kernel_size , primary_dilation, secondry_dilation , reciver  ) :

    g = reciver 
    g_4 = layers.Conv3DTranspose(primary_filters, primary_kernel_size, strides=(2, 2, 2), padding="same",dilation_rate = primary_dilation, activation='relu')(g)
    g_4 = layers.Conv3D(primary_filters, primary_kernel_size, padding='same', dilation_rate = primary_dilation ,kernel_initializer='he_normal')(g_4)
    g_4 = BatchNormalization()(g_4)
    g_4x = layers.Conv3DTranspose(secondry_filters, secondry_kernel_size, strides=(2, 2, 2), padding="same",dilation_rate = secondry_dilation, activation='relu')(g)
    g_4x = layers.Conv3D(secondry_filters , secondry_kernel_size , padding='same', dilation_rate = secondry_dilation ,kernel_initializer='he_normal')(g_4x)
    g_4x = BatchNormalization()(g_4x)
    g_5x = layers.concatenate([g_4x,g_4])
    return g_5x






def nucleus (reciver,filters ):
    x = reciver
    x = layers.Conv3D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = layers.Conv3D(filters, 3, padding='same', dilation_rate=(2, 2, 2), kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = keras.layers.Add()([ reciver , x])
    x = layers.Activation('relu')(x)
    y = layers.Conv3D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    y = layers.Conv3D(filters, 3, padding='same', dilation_rate=(2, 2, 2),kernel_initializer='he_normal')(y)
    y = BatchNormalization()(y)
    y = keras.layers.Add()([ x , y])
    y = layers.Activation('relu')(y)
    y = layers.Dropout(0.1)(y)
    return y


class spatial_attention (layers.Layer):
     def __init__(self):
         super().__init__()
         self.conv = layers.Conv3D(1, (3,3,3) ,  strides=(1, 1, 1), padding="same",dilation_rate=(2, 2, 2) , activation='sigmoid')
     def call (self,x):
         y = tf.reduce_max(x, axis=-1, keepdims=True)     
         z = tf.reduce_mean(x, axis=-1, keepdims=True)
         w = tf.keras.layers.concatenate([y,z])
         attention = self.conv(w)

         return x* attention


                                    


In [6]:


image_input = Input(shape=(112,112,80 , 2 ),name = 'image_input')
    
nasal = sensory_neuron(32,64,3,(1,1,1),image_input)
nasal_nucleus = nucleus (nasal,64 )
nasal_deeplayerB = spatial_attention()( nasal_nucleus)
    
temporal = sensory_neuron(32,64,5,(1,1,1),image_input)
temporal_nucleus = nucleus (temporal,64)
temporal_deeplayerB = spatial_attention()( temporal_nucleus)
    
crossover = layers.concatenate([nasal_deeplayerB,temporal_deeplayerB]) 
crossover_nucleus = nucleus (crossover,128)
bypassing_fiber = layers.Conv3DTranspose(128 , 3, strides=(2, 2, 2), padding="same", activation='relu')(crossover_nucleus)
nasotemporal_v1 = sensory_neuron(128,128 ,3,(1,1,1),crossover_nucleus)
nasotemporal_v2 = sensory_neuron(128,128 ,3,(2,2,2),crossover_nucleus)
opticnerve = layers.concatenate([nasotemporal_v1,nasotemporal_v2]) 
laterel_geniculate_body_v1 = laterel_geniculate_body (240 ,1 ,(1,1,1), opticnerve)
laterel_geniculate_body_v1 = nucleus (laterel_geniculate_body_v1,240 )
laterel_geniculate_body_v2 = laterel_geniculate_body (240 ,3 ,(1,1,1), opticnerve)
laterel_geniculate_body_v2 = nucleus (laterel_geniculate_body_v2,240 )
laterel_geniculate_body_v3 = laterel_geniculate_body (240 ,5 ,(1,1,1), opticnerve)
laterel_geniculate_body_v3 = nucleus (laterel_geniculate_body_v3,240)
optic_radiation = layers.concatenate([laterel_geniculate_body_v1,laterel_geniculate_body_v2,laterel_geniculate_body_v3])
motor_neuron_v1 = motor_neuron (240, 240 ,3,5 ,(1,1,1),(1,1,1) , optic_radiation  ) 
linker_v1 = layers.concatenate([motor_neuron_v1,opticnerve])
motor_neuron_v2 = motor_neuron (128, 128 ,3,5 ,(1,1,1),(1,1,1) , linker_v1  ) 
linker_v2 = layers.concatenate([motor_neuron_v2,crossover_nucleus])
motor_neuron_v3 = motor_neuron (64, 64 ,3,5 ,(1,1,1),(1,1,1) , linker_v2 )
linker_v3 = layers.concatenate([motor_neuron_v3,bypassing_fiber])
motor_neuron_v4 = layers.Conv3D(64, 3, padding='same', kernel_initializer='he_normal', activation='relu')(linker_v3)
motor_neuron_v4 = BatchNormalization()(motor_neuron_v4)
motor_neuron_v5 = layers.Conv3D(32, 3, padding='same', kernel_initializer='he_normal', activation='relu')(motor_neuron_v4)
motor_neuron_v5 = BatchNormalization()(motor_neuron_v5)
output = layers.Conv3D(4, (1,1,1) , padding="same", activation='softmax',name = 'mask_output')(motor_neuron_v5)

model = keras.Model( inputs=image_input, outputs=output )




In [7]:
model.load_weights("/kaggle/input/phase-4/trainingstepfourv2.weights.h5",skip_mismatch=True)

In [8]:


opti = keras.optimizers.Adam(learning_rate=(1e-5),name="adam")






    
model.compile(optimizer=opti,
    loss={
        'mask_output':combined_loss ,  

    },
    metrics={
        'mask_output': [iou],  

    })
    
    
model.fit(
    train_dataset,
    steps_per_epoch=220,             
    validation_data=val_dataset,
    validation_steps = 44 ,
    epochs=20)

Epoch 1/20


I0000 00:00:1756909382.910854      69 service.cc:148] XLA service 0x7c3d080804e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756909382.911900      69 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1756909387.742751      69 cuda_dnn.cc:529] Loaded cuDNN version 90300
E0000 00:00:1756909393.610858      69 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756909393.918483      69 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-09-03 14:23:15.012247: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=0,k3=0} for conv (f16[2,64,112,112,80]{4,3,2,1,0}, u8[0]{0}) custom-call(f16[2,32,112,112,80]{4,3,2,1,0}, f16[

220/220 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - iou: 0.7877 - loss: 0.3934

E0000 00:00:1756911293.566974      69 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1756911293.873358      69 gpu_timer.cc:82] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-09-03 14:54:54.965092: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng11{k2=0,k3=0} for conv (f16[2,64,112,112,80]{4,3,2,1,0}, u8[0]{0}) custom-call(f16[2,32,112,112,80]{4,3,2,1,0}, f16[64,32,5,5,5]{4,3,2,1,0}, f16[64]{0}), window={size=5x5x5 pad=2_2x2_2x2_2}, dim_labels=bf012_oi012->bf012, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues

220/220 ━━━━━━━━━━━━━━━━━━━━ 2324s 8s/step - iou: 0.7877 - loss: 0.3933 - val_iou: 0.7369 - val_loss: 0.3998
Epoch 2/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 1798s 8s/step - iou: 0.8066 - loss: 0.3733 - val_iou: 0.7472 - val_loss: 0.4056
Epoch 3/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 1798s 8s/step - iou: 0.7910 - loss: 0.3763 - val_iou: 0.7389 - val_loss: 0.4005
Epoch 4/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 1799s 8s/step - iou: 0.8083 - loss: 0.3529 - val_iou: 0.7326 - val_loss: 0.4043
Epoch 5/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 1798s 8s/step - iou: 0.8035 - loss: 0.3409 - val_iou: 0.7316 - val_loss: 0.4070
Epoch 6/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 1798s 8s/step - iou: 0.7990 - loss: 0.3619 - val_iou: 0.7313 - val_loss: 0.4030
Epoch 7/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 1799s 8s/step - iou: 0.8067 - loss: 0.3946 - val_iou: 0.7323 - val_loss: 0.4044
Epoch 8/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 1798s 8s/step - iou: 0.7988 - loss: 0.3614 - val_iou: 0.7228 - val_loss: 0.4143
Epoch 9/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 1798s 8s/st

In [9]:
model.save_weights("/kaggle/working/segmentationtrainingv1.weights.h5")